In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import datetime as dt

import os

### Read me: data processing methods


#### _Data:

The original data is located in the 'Data' folder. For 2016 and 2017 each row returns the colony counts of all plates for one medium at one location. In the 2020 data each row returns the colony counts for one plate from one medium for one sample day.

#### _Mediums_

Mediums for 2016 and 2017 data: Only use rows with easy_gel medium for bioindicator scores. These can be compared directly to 2020 levine bioindicator scores: 'green_met'.

This workbook translates all bioindicator scores to 'green_met' for 2016 and 2017.

#### _Bioindicator colonies:_

The column names for the bioindicators:

**2016-2017**

1. P1_24h_big_blue, P2_24h_big_blue, P3_24h_big_blue
2. 'P1_fluo_halo_colonies', 'P2_fluo_halo_colonies', 'p3_fluo_halo_colonies'



**2020**

1. green_met

#### _Aggregating non bioindicator colonies:_

Non bioindicator colonies can be added as a group to the total colony count.


#### _Volumes/qty of sample:_

All volumes/quantities are in ml and need to be normalized to 100ml. Example if the quantity given is 4ml then multiply that value by 25, if the volume is 0.2 multiply times 500.

#### _Definition of total colony count:_

The total colony count is the combined total of all reported colony counts for one sample day and location, this includes all colony categories.


#### _Reporting:_

Colony counts are reported in number of colony forming units per 100ml of sample.

1. Bioindicator median/average per location for each week of sampling
2. The upper 90‑percentile of the probability density function. See page 12 of the [DIRECTIVE 2006/7/EC](http://wiki.hackuarium.ch/images/8/8c/DIRECTIVE2006EC_.pdf), for details on calculating.
3. Confidence intervals for all reported values, correlation co-eficients.
4. Statistical significance of colony counts with and without Jazz
5. Probability of having repeated pollution events


**Rain data**

Rain is reported in mm/day, Starting the first day of sampling.

**Hotel nights**

The hotel nights sold is derived from [Stat-tab](https://www.pxweb.bfs.admin.ch/pxweb/en/)

#### _Handling Nan:_

Depends on the medium and the value of TMTC:

**If the medium is LB and TMTC is true or false:**
1. Use the 'other' value
2. The color category column values are N/A

**If the medium is levine and colony is bio-indicator and TMTC is True:**

1. Use the green_met value
2. Colored column values are estimates and added up under the 'other' column

**If the medium is levine and colony is bio-indicator and TMTC is False:**

1. Use the green_met value
2. All row values are valid

#### Corrections and criteria:

1. Scale x-axis evenly for all charts
2. Use term bioindicator instead of colony categories
3. Reduce the number of colors in charts
4. Spacing inside charts
5. Image format: .jpeg, .png
6. No titles but label x and y axis, include legend and units.

In [2]:
here = os.getcwd()
data = "{}/data".format(here)
translator = '{}/JSON/translator.json'.format(data)

# import colony counts
cc2016 = pd.read_csv("{}/2016_Data.csv".format(data))
cc2017 = pd.read_csv("{}/2017_Data.csv".format(data))

# the 2016 needs these columns added 
cc2016['medium'] = 'easy_gel'
cc2016['comments'] = None

# consider using rule to apply N/A where appropriate
cc2020 = pd.read_csv("{}/CSV/2020_Data.csv".format(data), encoding='latin1')
cc2020 = cc2020[['Date', 'Location', 'medium', 'Plate_number', 'Image', 'volume','green_met', 'purple', 'mauve', 'pink', 'other','tmtc', 'comments']].copy()
cc2020.tmtc.fillna(False, inplace=True)
cc2020.fillna(0, inplace=True)

# import rain data
rd20  = '{}/rain2020.csv'.format(data)
rain2020 = pd.read_csv(rd20)

rd16 = '{}/CSV/rain2016.csv'.format(data)
rain2016 = pd.read_csv(rd16)

rd17 = '{}/CSV/rain2017.csv'.format(data)
rain2017 = pd.read_csv(rd17)


with open(translator) as a_file:
    trans = json.load(a_file)


# the translator
print(trans)

{'green_met': ['big_blue', 'fluo_halo'], 'pink': ['pink'], 'purple': ['medium_blue'], 'other': ['other'], 'mauve': ['green', 'turquoise']}


In [3]:
# bio_indicators
big_blue = ['P1_24h_big_blue', 'P2_24h_big_blue', 'P3_24h_big_blue']
fluo_halo = ['P1_fluo_halo_colonies', 'P2_fluo_halo_colonies', 'p3_fluo_halo_colonies']

# colony counts not bioindicators
pink = [F'P{x}_24h_pink' for x in [1, 2, 3] ]
purple = [F'P{x}_24h_med_blue' for x in [1, 2, 3]]
other = [F'P{x}_24h_other' for x in [1, 2, 3]]
mauve = [F'P{x}_24h_green' for x in [1, 2, 3]]
mauve_2 = [F'P{x}_24h_turq' for x in [1, 2, 3]]

# the comments for each plate in the 2017 data
comments = ['Comments_p1_24h', 'Comments_p2_24h', 'Comments_p3_24h']


plate_one = [x[0] for x in [big_blue,fluo_halo, pink, purple, other, mauve, mauve_2]]
plate_two = [x[1] for x in [big_blue,fluo_halo, pink, purple, other, mauve, mauve_2]]
plate_three = [x[2] for x in [big_blue,fluo_halo, pink, purple, other, mauve, mauve_2]]

# locations of interest
locations = ['MRD', 'SVT', 'VNX']
mediums = ['easy_gel', 'levine']

In [4]:
# get only the locations and mediums of interest
# this makes an array of dictionaries, one for each record
cc2017_dict = cc2017.loc[(cc2017.Location.isin(locations))&(cc2017.medium.isin(mediums))].to_dict(orient='records')
cc2016_dict = cc2016.loc[(cc2016.Location.isin(locations))].to_dict(orient='records')

In [15]:
# columns from the 2020 data
colkeys = [
    'Date',
    'Location',
    'medium',
    'Plate_number',
    'Image',
    'volume',
    'green_met',
    'purple',
    'mauve',
    'pink',
    'other',
    'tmtc',
    'comments'
]
def make_results_array(results, colkeys, comments, sixteen=False):
    new_res = []
    for res in results:
        for i,records in enumerate([plate_one, plate_two, plate_three]):
            a_dict =  {x:None for x in colkeys}
            if sixteen:
                a = ma_p_values_to_202016(res, a_dict, records, i, comments)
            else:
                a = ma_p_values_to_2020(res, a_dict, records, i, comments)            
            new_res.append(a)
    return new_res       

# maps values from 2017 to new dict:
def ma_p_values_to_2020(res, a_dict, records, i, comments):    
    a_dict.update({'Location':res['Location']})
    a_dict.update({'Date':res['Date']})
    a_dict.update({'Plate_number':i+1})
    a_dict.update({'medium':res['medium']})
    a_dict.update({'volume':res['P1_qty_sample']})
    a_dict.update({'comments':res[comments[i]]})
    a_dict.update({'green_met':res[records[0]]+res[records[1]]})
    a_dict.update({'pink':res[records[2]]})
    a_dict.update({'purple':res[records[3]]})
    a_dict.update({'mauve':res[records[5]]+res[records[6]]})
    a_dict.update({'other':res[records[4]]})        
    return a_dict

# the column names and comments do not match the records from 2017:
def ma_p_values_to_202016(res, a_dict, records, i, comments):    
    a_dict.update({'Location':res['Location']})
    a_dict.update({'Date':res['Date']})
    a_dict.update({'Plate_number':i+1})
    a_dict.update({'medium':res['medium']})
    a_dict.update({'volume':res['P1_qty_sample']})
    a_dict.update({'comments':res['comments']})
    a_dict.update({'green_met':res[records[0]]})
    a_dict.update({'pink':res[records[2]]})
    a_dict.update({'purple':res[records[3]]})
    a_dict.update({'mauve':res[records[6]]})
    a_dict.update({'other':res[records[4]]})        
    return a_dict
                
new2017 = make_results_array(cc2017_dict,colkeys, comments)
new2016 = make_results_array(cc2016_dict,colkeys, comments, sixteen=True)
    
newdf2017 = pd.DataFrame(new2017)
newdf2017['Date'] = newdf2017.Date.apply(lambda x: dt.datetime.strptime(x, '%d.%m.%y').strftime('%Y-%m-%d'))
newdf2016 = pd.DataFrame(new2016)


In [6]:
cc2020

,Date,Location,medium,Plate_number,Image,volume,green_met,purple,mauve,pink,other,tmtc,comments
0,2020-06-11,Echandens,LB,1.0,IMG_9308.JPG,0.2,0.0,0.0,0.0,0.0,1000.0,True,"TMTC really, at least 4 main colony types"
1,2020-06-11,Echandens,levine,1.0,IMG_9309.JPG,0.2,50.0,0.0,0.0,0.0,1000.0,True,TMTC for all
2,2020-06-11,VNX,LB,1.0,IMG_9310.JPG,1.0,0.0,0.0,0.0,0.0,23.0,False,3 types - cut in agar
3,2020-06-11,VNX,levine,1.0,IMG_9311.JPG,1.0,0.0,25.0,7.0,0.0,0.0,False,"plates too wetÉ smears - lawn basically, but a..."
4,2020-06-11,VNX,levine,2.0,IMG_9312.JPG,1.0,0.0,8.0,7.0,0.0,0.0,False,big smear with more purple too
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,2020-08-06,MRD,levine,2.0,IMG_1010.JPG,1.0,0.0,3.0,5.0,7.0,21.0,False,0
145,2020-08-06,MRD,levine,3.0,IMG_1011.JPG,1.0,0.0,2.0,4.0,5.0,23.0,False,0
146,2020-08-06,Fountain@Sauvetage,LB,1.0,IMG_1005.JPG,1.0,0.0,0.0,0.0,0.0,0.0,False,0
147,2020-08-06,Fountain@Sauvetage,levine,1.0,IMG_1006.JPG,1.0,0.0,0.0,0.0,0.0,0.0,False,0


In [7]:
newdf2016 

,Date,Location,medium,Plate_number,Image,volume,green_met,purple,mauve,pink,other,tmtc,comments
0,2016-07-05,MRD,easy_gel,1,None,0.4,0,0,0,0,0,None,None
1,2016-07-05,MRD,easy_gel,2,None,0.4,0,1,0,0,0,None,None
2,2016-07-05,MRD,easy_gel,3,None,0.4,0,1,0,0,0,None,None
3,2016-07-12,MRD,easy_gel,1,None,4.0,22,0,0,20,37,None,None
4,2016-07-12,MRD,easy_gel,2,None,4.0,17,3,4,17,26,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2016-08-09,VNX,easy_gel,2,None,4.0,13,19,9,0,7,None,None
68,2016-08-09,VNX,easy_gel,3,None,4.0,14,5,0,4,5,None,None
69,2016-08-02,VNX,easy_gel,1,None,4.0,3,4,0,1,0,None,None
70,2016-08-02,VNX,easy_gel,2,None,4.0,2,0,0,1,2,None,None


In [14]:
newdf2017

,Date,Location,medium,Plate_number,Image,volume,green_met,purple,mauve,pink,other,tmtc,comments
0,2017-06-12,SVT,easy_gel,1,None,4.0,0.0,0.0,0.0,0.0,0.0,None,No comments
1,2017-06-12,SVT,easy_gel,2,None,4.0,0.0,0.0,0.0,0.0,0.0,None,No comments
2,2017-06-12,SVT,easy_gel,3,None,4.0,0.0,0.0,0.0,0.0,0.0,None,No comments
3,2017-06-12,VNX,easy_gel,1,None,4.0,0.0,0.0,0.0,0.0,0.0,None,No comments
4,2017-06-12,VNX,easy_gel,2,None,4.0,0.0,0.0,0.0,0.0,0.0,None,No comments
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2017-07-03,MRD,levine,2,None,0.5,NaN,NaN,NaN,NaN,NaN,None,NaN
80,2017-07-03,MRD,levine,3,None,0.5,NaN,NaN,NaN,NaN,NaN,None,NaN
81,2017-07-03,SVT,levine,1,None,0.5,0.0,3.0,0.0,0.0,1.0,None,"medium blue=purple colonies, coliform (no meta..."
82,2017-07-03,SVT,levine,2,None,0.5,0.0,0.0,0.0,0.0,1.0,None,No comments
